# Label Creation - Abnormal Returns

In [1]:
import pandas as pd
import numpy as np

RAW_DATA_FOLDER_AR = "../../data_untracked/raw/abnormal_returns" 
PROCESSED_DATA_FOLDER = "../../data_untracked/processed"

# Change DEBUG to False to see less printouts
DEBUG = True
if DEBUG:
    import pandas_market_calendars as mcal
    nyse = mcal.get_calendar("NYSE") # Get the NYSE trading calendar to calculate number of trading days to check validility of CRSP data

## Load required datasets
1. `all_transactions_final`: output of `sec_data_merging.ipynb`
2. `all_beta_daily`: beta and return values from crsp data for all tickers 
3. `risk_free_rate_daily`: risk free rate from 2005 to 2021, downloaded from https://fred.stlouisfed.org/series/DTB3
    * This rate is the same for ALL tickers, joined on the transaction date
4. `unique_ticker_trans_8above`: unique list of tickers with more than 8 transactions from Form 4 merged data
    * This list is to create abnormal return calculations for each ticker, in order for computational efficiency
5. **Note that** `stock_price`: is the daily stock price for each ticker, loaded for each ticker in abnormal return calculation because the data files are stored as '<ticker>.csv'

In [2]:
# This loads the output of sec_data_merging.ipynb merging notebook
all_transactions_final = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/all_transactions_merged.csv")
all_transactions_final['TRANS_DATE'] = pd.to_datetime(all_transactions_final['TRANS_DATE'], errors='coerce')
all_transactions_final.dropna(subset=['TRANS_PRICEPERSHARE'], inplace=True)
all_transactions_final.shape # (2546985, 24)

(2254197, 24)

In [22]:
# This loads the beta and return values from crsp data
all_beta_daily = pd.read_csv(f"{RAW_DATA_FOLDER_AR}/crsp/beta_daily.csv")
all_beta_daily['DATE'] = pd.to_datetime(all_beta_daily['DATE'], errors='coerce')
all_beta_daily.rename(columns={'RET':'ret_beta'}, inplace=True) # rename RET from beta dataset to ret_beta to avoid confusion
all_beta_daily.shape # (22639404, 14)

(22639404, 14)

In [29]:
# This loads the risk free rate from 2005 to 2021, downloaded from https://fred.stlouisfed.org/series/DTB3 
risk_free_rate_daily = pd.read_csv(f"{RAW_DATA_FOLDER_AR}/fred/risk_free_rate_daily.csv")
risk_free_rate_daily.rename(columns={'observation_date':'date', 'DTB3':'risk_free_rate'}, inplace=True)
risk_free_rate_daily['date'] = pd.to_datetime(risk_free_rate_daily['date'], errors='coerce')
risk_free_rate_daily.shape # (4435, 2)

(6523, 2)

In [6]:
# This loads the unique list of tickers with more than 8 transactions from Form 4 merged data
unique_ticker_trans_8above = pd.read_csv(f'{PROCESSED_DATA_FOLDER}/unique_names_trans_8above.csv')
ticker_list = unique_ticker_trans_8above[['ISSUERTRADINGSYMBOL']].drop_duplicates().values.flatten()

## Define hyperparameters for CAR
- this can be modified based on literature reviews and our EDA

In [10]:
DAYS_BEFORE_TRANSACTION = 5
DAYS_AFTER_TRANSACTION = 0
MIN_AR_VALUES = 3

## Define helper functions

In [30]:
# Function to impute categorical numerical column where NA values are between the same values
def categorical_impute(series):
    """
    Imputes missing values by carrying forward the same value if the before and after values are identical.
    Otherwise, leaves NaN.
    """
    series = series.copy()  # Avoid modifying original data
    for i in range(1, len(series) - 1):
        if pd.isna(series[i]):  # If current value is NaN
            prev_idx = i - 1
            next_idx = i + 1

            # Find the next non-NaN value
            while next_idx < len(series) and pd.isna(series[next_idx]):
                next_idx += 1

            # If both a previous and next valid value exist and they are the same, impute
            if prev_idx >= 0 and next_idx < len(series) and series[prev_idx] == series[next_idx]:
                series[i] = series[prev_idx]

    return series

# Function to perform mean imputation for consecutive NaN values
def mean_impute_between_values(series):
    """
    Imputes missing values by taking the mean of the nearest non-NaN values before and after.
    """
    series = series.copy()  # Avoid modifying original data
    for i in range(1, len(series) - 1):
        if pd.isna(series[i]):  # If current value is NaN
            prev_idx = i - 1
            next_idx = i + 1

            # Find the next non-NaN value
            while next_idx < len(series) and pd.isna(series[next_idx]):
                next_idx += 1

            # If both a previous and next valid value exist, take their mean
            if prev_idx >= 0 and next_idx < len(series) and not pd.isna(series[prev_idx]) and not pd.isna(series[next_idx]):
                series[i] = (series[prev_idx] + series[next_idx]) / 2

    return series

## Compute cumulative abnormal return (CAR) within the given window
def calculate_cumulative_abnormal_return(trans_date, permno, abnormal_ret_data, 
                                         days_before = DAYS_BEFORE_TRANSACTION, 
                                         days_after = DAYS_AFTER_TRANSACTION, 
                                         min_ar_values = MIN_AR_VALUES):
    '''
    This is meant to be used in a .apply() function, for each row of transaction data. 
    For each transaction date, retrieve the PERMNO and find a subset of abnormal_returns during the event window, and sum abnormal_ret to get CAR
    '''
    # Define the event window (-6 to +2 days)
    start_date = trans_date - pd.Timedelta(days=days_before)
    end_date = trans_date + pd.Timedelta(days=days_after)

    # Filter abnormal return data based on date range and matching PERMNO
    subset = abnormal_ret_data[(abnormal_ret_data['date'] >= start_date) & 
                            (abnormal_ret_data['date'] <= end_date) & 
                            (abnormal_ret_data['PERMNO'] == permno)]

    # Ensure at least X non-NA values before summing
    count = subset['abnormal_ret'].count()
    if count >= min_ar_values:
        ### Initially wanted to also return standard deviation / Z score
        #std = np.std(subset['abnormal_ret'])
        #if std == 0: # unlikely, but it may be possible that there is no change in abnormal_return over the time period
        #    t_test = None
        #else: 
        #    t_test = (subset['abnormal_ret'].sum()/count) / (np.std(subset['abnormal_ret']) / np.sqrt(count))

        return subset['abnormal_ret'].sum()
    
    else:

        return None # Return None if there aren't at least X valid values

In [31]:
# Main function to calculate abnormal returns
def update_trans_ticker_and_create_ar_compare(ticker:str, permno:float,trans_df_permno, stock_price_df, beta_ticker_df, 
                                              risk_free_rate_df=risk_free_rate_daily, days_before=DAYS_BEFORE_TRANSACTION, days_after=DAYS_AFTER_TRANSACTION):
    '''
    This is meant to be run for each (TICKER, PERMNO) pair

    Parameters:
    ticker: TICKER of the stock in string
    permno: PERMNO of the ticker in string
    trans_df_permno: DataFrame of transactions for the specific (ticker and) PERMNO. All permnos are the same in this dataframe
    stock_price_df: DataFrame of stock prices for (TICKER, PERMNO) pair
    beta_ticker_df: DataFrame of beta values for (TICKER, PERMNO) pair
    risk_free_rate_df: DataFrame of risk free rate values

    Returns:
    ticker_ar: DataFrame with date -6 and +2 of the permno's date range, with daily stock price, beta values, risk free rate and abnormal returns 
    trans_ticker_price: DataFrame of transactions with abnormal returns and cumulative abnormal returns based on hyperparameters (currently -6, +2)
    '''

    # Get start and end of the transaction date, to then create a range of abnormal returns
    start, end = str(min(trans_df_permno['TRANS_DATE']))[:10], str(max(trans_df_permno['TRANS_DATE']))[:10]

    ## The date range here can be modified!!!
    start_new, end_new = pd.to_datetime(start) - pd.to_timedelta(days_before, unit='d'), pd.to_datetime(end) + pd.to_timedelta(days_after, unit='d')

    # Create a new dataframe to store values associated with daily transactions and abnormal returns
    ticker_ar = pd.DataFrame({'date': pd.date_range(start=start_new, end=end_new)})
    ticker_ar['date'] = pd.to_datetime(ticker_ar['date'], errors='coerce')

    if DEBUG: ticker_ar_len = ticker_ar.shape[0]

    # Filter stock price data and beta data to match PERMNO
    stock_price_filtered = stock_price_df[(stock_price_df['PERMNO'] == permno)]
    beta_ticker_filtered = beta_ticker_df[(beta_ticker_df['PERMNO'] == permno)]
    
    # Merge date range with stock price, beta values and risk free rate
    ticker_ar = ticker_ar.merge(stock_price_filtered[['date', 'TICKER', 'RET', 'RETX', 'VOL']].drop_duplicates(), on='date', how='left')
    #if DEBUG and ticker_ar_len != ticker_ar.shape[0]: print(f"{ticker} Missing rows after merge with stock price: before {ticker_ar_len}, after {ticker_ar.shape[0]}")
    ticker_ar = ticker_ar.merge(beta_ticker_filtered[['DATE', 'PERMNO', 'ret_beta','alpha', 'b_mkt']], left_on='date', right_on='DATE', how='left')
    if DEBUG and abs(ticker_ar_len - ticker_ar.shape[0]) > 10: print(f"{permno} Missing rows after merge with beta: before {ticker_ar_len}, after {ticker_ar.shape[0]}")
    ticker_ar = ticker_ar.merge(risk_free_rate_df, on='date', how='left')

    # Add relevant columns for identification when merging to overall dataset
    ticker_ar['TICKER'] = ticker
    ticker_ar['PERMNO'] = permno

    # Transform actual_returns from beta data (object type) to float type
    ticker_ar['actual_ret'] = ticker_ar['ret_beta'].str.replace('%', '') #.astype(float, errors='coerce') / 100
    ticker_ar['actual_ret'] = pd.to_numeric(ticker_ar['actual_ret'], errors='coerce') / 100

    # Impute: if theres values IN BETWEEN non-NA values, then impute mean. Start and End NA values ignore (the most likely case is because there was no stock data yet)
    ticker_ar['actual_ret'] = mean_impute_between_values(ticker_ar['actual_ret'].copy())
    ticker_ar['b_mkt'] = mean_impute_between_values(ticker_ar['b_mkt'].copy())
    ticker_ar['risk_free_rate'] = mean_impute_between_values(ticker_ar['risk_free_rate'].copy())

    # Calculate expected return and abnormal return
    ticker_ar['expected_ret'] = ticker_ar['risk_free_rate'] + ticker_ar['b_mkt'] * (ticker_ar['actual_ret'] - ticker_ar['risk_free_rate'])
    ticker_ar['abnormal_ret'] = ticker_ar['actual_ret'] - ticker_ar['expected_ret']

    trans_ticker_price = trans_df_permno.merge(ticker_ar[['date','actual_ret','b_mkt','risk_free_rate','expected_ret','abnormal_ret']], left_on='TRANS_DATE', right_on='date', how='left')

    # Apply function to calculate CAR for [-5, 0] days from each transaction date (several days before)
    trans_ticker_price['CAR_5_before'] = trans_ticker_price.apply(
        lambda row: calculate_cumulative_abnormal_return(row['TRANS_DATE'], row['PERMNO'], ticker_ar,
                                                         5, 0, 4),
        axis=1#, result_type ='expand'
    )

    # Apply function to calculate CAR for [0, 5] days from each transaction date (several days after)
    trans_ticker_price['CAR_5_after'] = trans_ticker_price.apply(
        lambda row: calculate_cumulative_abnormal_return(row['TRANS_DATE'], row['PERMNO'], ticker_ar,
                                                         0, 5, 4),
        axis=1#, result_type ='expand'
    )

    # Apply function to calculate CAR for [-30, 0] days from each transaction date (a month before)
    trans_ticker_price['CAR_30_before'] = trans_ticker_price.apply(
        lambda row: calculate_cumulative_abnormal_return(row['TRANS_DATE'], row['PERMNO'], ticker_ar,
                                                         30, 0, 25),
        axis=1#, result_type ='expand'
    )

    # Apply function to calculate CAR for [0, 30] days from each transaction date (a month after)
    trans_ticker_price['CAR_30_after'] = trans_ticker_price.apply(
        lambda row: calculate_cumulative_abnormal_return(row['TRANS_DATE'], row['PERMNO'], ticker_ar,
                                                         0, 30, 25),
        axis=1#, result_type ='expand'
    )

    # Apply function to calculate CAR for [-60, 0] days from each transaction date (a month before)
    trans_ticker_price['CAR_60_before'] = trans_ticker_price.apply(
        lambda row: calculate_cumulative_abnormal_return(row['TRANS_DATE'], row['PERMNO'], ticker_ar,
                                                         60, 0, 45),
        axis=1#, result_type ='expand'
    )

    # Apply function to calculate CAR for [0, 60] days from each transaction date (a month after)
    trans_ticker_price['CAR_60_after'] = trans_ticker_price.apply(
        lambda row: calculate_cumulative_abnormal_return(row['TRANS_DATE'], row['PERMNO'], ticker_ar,
                                                         0, 60, 45),
        axis=1#, result_type ='expand'
    )

    # Apply function to calculate CAR for [120, 0] days from each transaction date (a month before)
    trans_ticker_price['CAR_120_before'] = trans_ticker_price.apply(
        lambda row: calculate_cumulative_abnormal_return(row['TRANS_DATE'], row['PERMNO'], ticker_ar,
                                                         120, 0, 100),
        axis=1#, result_type ='expand'
    )

    # Apply function to calculate CAR for [0, 120] days from each transaction date (a month after)
    trans_ticker_price['CAR_120_after'] = trans_ticker_price.apply(
        lambda row: calculate_cumulative_abnormal_return(row['TRANS_DATE'], row['PERMNO'], ticker_ar,
                                                         0, 120, 100),
        axis=1#, result_type ='expand'
    )

    return ticker_ar, trans_ticker_price 

## Create abnormal_ret column for each ticker
Steps for each ticker:

1. merge transaction data on the same date with the nearest average CRSP stock price (PRC), in order to get most likely PERMNO for the ticker
2. Based on number of unique PERMNOs matched, for each (date-disjoint PERMNO),
- call `update_trans_ticker_and_create_ar_compare` function to:
     1. Create a dataframe with `date` column matching -6 and +2 of the permno's date range (CAN BE MODIFIED BY HYPERPARAMETER)
     2. Based on date, get returns, beta and risk_free_rate from beta_daily and risk_free_rate_df

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
existing_time_frame = {}
count = 0
count_ticker = 0
tickers_multiple_permno = {2: {'disjoint':[], 'overlapping_dates':{'less_than_0.2':[],'more_than_0.2':[]}}, 
                           3: {'disjoint':[], 'overlapping_dates':{'less_than_0.2':[],'more_than_0.2':[]}}, 
                           4: {'disjoint':[], 'overlapping_dates':{'less_than_0.2':[],'more_than_0.2':[]}}}
ticker_not_found_price = []
ticker_not_found_transaction = []

# Stores the transactions data, with abnormal return columns
all_trans_ar = pd.DataFrame()
# Stores the daily stock price data, with abnormal return columns - this is for cumulative abnormal returns
all_ticker_ar_compare = pd.DataFrame()

for ticker in ticker_list:
    count_ticker += 1
    try: # just to check if the file exists 
        pd.read_csv(f"{RAW_DATA_FOLDER_AR}/crsp/daily_stock_data_by_ticker/{ticker}.csv",nrows=1)
    except FileNotFoundError:
        ticker_not_found_price.append(ticker)
        continue
    else:
        stock_price = pd.read_csv(f"{RAW_DATA_FOLDER_AR}/crsp/daily_stock_data_by_ticker/{ticker}.csv")
        stock_price = stock_price.dropna(subset=['PRC'])
        stock_price['date'] = pd.to_datetime(stock_price['date'], errors='coerce')


    trans_ticker = all_transactions_final[all_transactions_final['ISSUERTRADINGSYMBOL'] == ticker]

    # break if ticker not found in our filtered transaction data
    if trans_ticker.shape[0] == 0:
        ticker_not_found_transaction.append(ticker)
        continue
    
    # Filter stock price to match transaction date range
    start, end = str(min(trans_ticker['TRANS_DATE']))[:10], str(max(trans_ticker['TRANS_DATE']))[:10]
    stock_price = stock_price[(stock_price['date'] >= start) & (stock_price['date'] <= end)]

    if DEBUG: # commented out
        # Check that the date range has a multiple of trading days
        # Commented out because it doesn't really matter, as long as there is not alot of NAs (checked for FNCB with rows but 966, expected 2661 days) 
        '''if (start, end) not in existing_time_frame:
            trading_days = len(nyse.schedule(start_date=start, end_date=end))
            existing_time_frame[(start, end)] = trading_days
        else: trading_days = existing_time_frame[(start, end)]

        if stock_price.shape[0] % trading_days != 0:
            print(f"Missing trading days for {ticker} {stock_price.shape[0]}, expected {trading_days}")
        '''
        #print(ticker[0], stock_price.shape)
        
    #### 1. Merge transaction data on the same date with the nearest average CRSP stock price (PRC), in order to get most likely PERMNO for the ticker
    # Note that merge_asof is a left join. NA values will be introduced in STOCK_PRICE_COLUMNS if there is no matching date, which happens.
    trans_ticker_price = pd.merge_asof(trans_ticker.sort_values('TRANS_PRICEPERSHARE'), stock_price[['PERMNO','date','VOL','PRC','RET','TICKER']].sort_values('PRC'), 
                                    left_on='TRANS_PRICEPERSHARE', right_on='PRC', left_by='TRANS_DATE', right_by='date', direction='nearest')
    
    # Apply categorical imputation for PERMNO, because stock_price will have NA values (or missing matching dates) for non-trading days. Whereas those days exist in transaction data
    # Impute PERMNO by carrying forward the same value if the before and after values are identical
    trans_ticker_price['PERMNO'] = categorical_impute(trans_ticker_price['PERMNO'])
    # Consider if we also want to impute volume?

    if DEBUG: # Print statement if mismatch rows after merging
        if trans_ticker_price.shape[0] != trans_ticker.shape[0]: print(f"Mismatch in rows after merge for {ticker}: before {trans_ticker.shape[0]}, after {trans_ticker_price.shape[0]}")
    
    beta_ticker = all_beta_daily[all_beta_daily['TICKER'] == ticker]
    # Check how many unique permcos for each ticker exist in our transaction data
    permno_count_dict = trans_ticker_price['PERMNO'].value_counts().to_dict() 
    permno_unique = len(permno_count_dict)

    if permno_unique == 1:
        # run function to calculate abnormal returns in the date range for unique permno
        permno = list(permno_count_dict.keys())[0]
        ticker_ar, trans_ticker_price = update_trans_ticker_and_create_ar_compare(ticker, permno, trans_ticker_price[trans_ticker_price['PERMNO'] == permno], stock_price, beta_ticker)
        # Append to the overall dataframe
        all_ticker_ar_compare = pd.concat([all_ticker_ar_compare, ticker_ar], axis=0)
        all_trans_ar = pd.concat([all_trans_ar, trans_ticker_price], axis=0)

    elif permno_unique > 1:
        # Check id date ranges for each unqiue permno are disjoint
        permno_ranges = trans_ticker_price.groupby('PERMNO')['TRANS_DATE'].agg(['min', 'max']).reset_index()
        is_disjoint = True
        sorted_ranges = permno_ranges.sort_values(by='min').reset_index(drop=True)
        for i in range(len(sorted_ranges) - 1):
            if sorted_ranges.loc[i, 'max'] >= sorted_ranges.loc[i + 1, 'min']:
                is_disjoint = False
                break

        if is_disjoint:
            # Update this in dictionary to keep track 
            tickers_multiple_permno[permno_unique]['disjoint'].append(ticker)

            # For each unique permno, create abnormal returns comparison and CAR for transaction as per usual
            for permno, count in permno_count_dict.items():
                # run function to calculate abnormal returns in the date range for each permno
                ## Important to have X_new here otherwise will overwrite 
                ticker_ar_new, trans_ticker_price_new = update_trans_ticker_and_create_ar_compare(ticker, permno, trans_ticker_price[trans_ticker_price['PERMNO'] == permno], stock_price, beta_ticker)
                # Append to the overall dataframe
                all_ticker_ar_compare = pd.concat([all_ticker_ar_compare, ticker_ar_new], axis=0)
                all_trans_ar = pd.concat([all_trans_ar, trans_ticker_price_new], axis=0)
        else:
            # Count the number of rows per PERMNO
            permno_counts = trans_ticker_price['PERMNO'].value_counts()
            min_permno, max_permno = permno_counts.idxmin(), permno_counts.idxmax() # get permno of the least and most amount of rows

            # Check if the smaller PERMNO is less than 20% of total rows
            if permno_counts[min_permno] / len(trans_ticker_price) <= 0.20:
                # Update this in dictionary to keep track
                tickers_multiple_permno[permno_unique]['overlapping_dates']['less_than_0.2'].append(ticker)

                # Not sure if we should assume them as same value..
                '''# Replace all occurrences of the lesser PERMNO with the larger one
                trans_ticker_price['PERMNO'] = trans_ticker_price['PERMNO'].replace(min_permno, max_permno) ### NOTE THAT HERE THE VOL AND RET RESULTS WOULD THEN BE WRONG...

                # Run function as per normal
                if permno_unique == 2:
                    # run function to calculate abnormal returns in the date range for each permno
                    ticker_ar, trans_ticker_price = update_trans_ticker_and_create_ar_compare(ticker, max_permno, trans_ticker_price[trans_ticker_price['PERMNO'] == max_permno], stock_price, beta_ticker)
                    # Append to the overall dataframe
                    all_ticker_ar_compare = pd.concat([all_ticker_ar_compare, ticker_ar], axis=0)
                    all_trans_ar = pd.concat([all_trans_ar, trans_ticker_price], axis=0)'''
            
            else:
                ## Note that overlapping time permno with more than 2 are simply excluded
                tickers_multiple_permno[permno_unique]['overlapping_dates']['more_than_0.2'].append(ticker)

print(f"Tickers with multiple permnos: {tickers_multiple_permno}")
print(f"Tickers not found in price data: {ticker_not_found_price}")
print(f"Tickers not found in transaction data: {ticker_not_found_transaction}")
print(f"Total {count_ticker} tickers processed")
## 300 cases took 7 minutes for Emily...

Tickers with multiple permnos: {2: {'disjoint': [], 'overlapping_dates': {'less_than_0.2': [], 'more_than_0.2': []}}, 3: {'disjoint': [], 'overlapping_dates': {'less_than_0.2': [], 'more_than_0.2': []}}, 4: {'disjoint': [], 'overlapping_dates': {'less_than_0.2': [], 'more_than_0.2': []}}}
Tickers not found in price data: []
Tickers not found in transaction data: []
Total 3 tickers processed


## Simple checking of data before downloading to local

In [33]:
all_trans_ar['abnormal_ret'].describe()

count    1338.000000
mean       -0.007387
std         0.204393
min        -1.717498
25%        -0.012362
50%        -0.005021
75%        -0.000599
max         0.663414
Name: abnormal_ret, dtype: float64

In [34]:
all_trans_ar['CAR_120_before'].describe()

count    1328.000000
mean       -0.641955
std        25.072193
min      -197.676295
25%        -1.116769
50%        -0.747213
75%        -0.237183
max        70.347152
Name: CAR_120_before, dtype: float64

In [35]:
all_trans_ar['CAR_5_after'].isna().sum()

21

In [36]:
print(all_trans_ar.shape)
print(all_ticker_ar_compare.shape)

(1347, 44)
(8693, 14)


In [37]:
all_ticker_ar_compare.head(2)

,date,TICKER,RET,RETX,VOL,DATE,PERMNO,ret_beta,alpha,b_mkt,risk_free_rate,actual_ret,expected_ret,abnormal_ret
0,2010-02-12,OMCL,NaN,NaN,NaN,2010-02-12,89110.0,0.3110%,0.0,0.7915,0.1,0.00311,0.023312,-0.020202
1,2010-02-13,OMCL,NaN,NaN,NaN,NaT,89110.0,NaN,NaN,0.7886,0.1,0.00078,0.021755,-0.020975


## To download data to local

In [38]:
## All transactions with abnormal return (and CAR)
all_trans_ar.to_csv(f"{PROCESSED_DATA_FOLDER}/transactions_abnormal_returns_{count_ticker}_ticker.csv")

## All tickers (unique PERMNO) with abnormal returns for the benchmark calculations 
## This dataset is independent of transactions, and should be used whenc calculating abnormal returns. Save to csv for future refence.
all_ticker_ar_compare.to_csv(f"{RAW_DATA_FOLDER_AR}/transactions_abnormal_returns_{count_ticker}_ticker_COMPARE.csv")

## Example

In [ ]:
all_ticker_ar_compare[(all_ticker_ar_compare['TICKER']=='OMCL') & (all_ticker_ar_compare['date']<='2018-03-08')& (all_ticker_ar_compare['date']>='2018-02-28')]['abnormal_ret'].sum()

5.095842877375001

In [ ]:
all_trans_ar[(all_trans_ar['TICKER']=='OMCL') & (all_trans_ar['TRANS_DATE']=='2018-03-06')]

,Unnamed: 0,TRANS_SK,ACCESSION_NUMBER,SECURITY_TITLE,TRANS_DATE,DEEMED_EXECUTION_DATE,TRANS_CODE,EQUITY_SWAP_INVOLVED,TRANS_TIMELINESS,TRANS_SHARES,TRANS_PRICEPERSHARE,TRANS_ACQUIRED_DISP_CD,SHRS_OWND_FOLWNG_TRANS,DIRECT_INDIRECT_OWNERSHIP,NATURE_OF_OWNERSHIP,trans_amt,FILING_DATE,PERIOD_OF_REPORT,ISSUERCIK,ISSUERNAME,ISSUERTRADINGSYMBOL,RPTOWNERCIK,RPTOWNERNAME,RPTOWNER_RELATIONSHIP,PERMNO,date_x,VOL,PRC,RET,TICKER,date_y,actual_ret,b_mkt,risk_free_rate,expected_ret,abnormal_ret,CAR
973,1684062,1002422,0001179110-18-003804,Common Stock,2018-03-06,NaN,S,0,NaN,8186.0,44.21,D,211323.0,D,NaN,361903.06,2018-03-07,2018-03-06,926326,"OMNICELL, Inc",OMCL,1227556,LIPPS RANDALL A,"Director,Officer",89110.0,2018-03-06,330136.0,44.75,0.019362,OMCL,2018-03-06,0.019362,1.3569,1.65,-0.562613,0.581975,5.095843
